In [1]:
import numpy as np
import os
from os import listdir
import librosa as lb
import pandas as pd
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import glob
import random
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torch
import torchaudio

In [2]:
if os.path.exists('data')==False:
    os.mkdir('data')

## Making a Experiment Folder

In [3]:
exp = 'exp-13'

In [4]:
if os.path.exists(os.path.join('data',exp))==False:
    os.mkdir(os.path.join('data',exp))
else:
    print('Folder is exists')

Folder is exists


In [5]:
dataset = 'dataset/Merge-Data/'

## Noise Augmentation

In [6]:
noisetypes = ['noise']
musan_path = 'dataset/Musan-Data'

noisesnr   = {'noise':[0,15],'speech':[13,20],'music':[5,15]}
numnoise   = {'noise':[1,1], 'speech':[3,7],  'music':[1,1] }
noiselist  = {}
augment_files   = glob.glob(os.path.join(musan_path,'*/*/*.wav'));

for file in augment_files:
    if not file.split('/')[-3] in noiselist:
        noiselist[file.split('/')[-3]] = []
    noiselist[file.split('/')[-3]].append(file)

In [7]:
def additive_noise(noisetypes, noiselist, noisesnr, numnoise, audio, noisecat=None):
    if noisecat is None:
        noisecat = noisetypes[0]

    wav = audio.reshape(audio.shape[-1])
    clean_db = 10 * np.log10(np.mean(wav ** 2)+1e-4) 
    numnoise = numnoise[noisecat]
    noiselist = random.sample(noiselist[noisecat], random.randint(numnoise[0],numnoise[1]))

    final_noise_audio = 0
    for noise in noiselist:
        noise_wav,sr = lb.load(noise)
        noise_wav = noise_wav.reshape(noise_wav.shape[-1])
        noise_wav_len = noise_wav.shape[0]
        wav_len = wav.shape[0]
        if noise_wav_len <= wav_len:
            continue

        noise_start_slice = random.randint(0,noise_wav_len-(wav_len+1))
        noise_wav = noise_wav[noise_start_slice:noise_start_slice+wav_len]

        noise_snr = random.uniform(noisesnr[noisecat][0],noisesnr[noisecat][1])
        noise_db = 10 * np.log10(np.mean(noise_wav ** 2)+1e-4)
        final_noise_audio = np.sqrt(10 ** ((clean_db - noise_db - noise_snr) / 10)) * noise_wav

    out_audio = np.array(wav + final_noise_audio)
    out_audio = out_audio.reshape(-1)
    return out_audio

## Normalization

In [8]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

In [9]:
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def normalization(sound,sr,scaler):
    sound = sound.reshape(sound.shape[0])
    
    inputs = processor(sound, sampling_rate=sr, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    out = outputs.last_hidden_state
    out = out.numpy().reshape(out.shape[1],out.shape[2])
    norm = scaler.fit_transform(out)
    
    return out, norm

In [11]:
scaler = StandardScaler()

## Extracting Train Data Feature

In [12]:
df_train = pd.read_csv('csv_files/experiment_data/train.csv')
df_train.head()

,filename,label
0,aHJPsm6esHeW5apkd8KAFdia9zj2.wav,positive
1,ptuwF1pj4ibIoVuZmt4nKb5naiZ2.wav,positive
2,EbWP0XeMpnbrTsljro0dOGKIell1_heavy.wav,positive
3,Ued1DKV2NkUnzWjJcPMxA6Q261r1_heavy.wav,positive
4,ZaeW7vuikQgf7984JVqXYOtUQMs2.wav,positive


In [13]:
img_train = []
label_train = []

to_hot_one = {'positive':0, 'negative':1}

for index,row in df_train.iterrows():
    name = row['filename']
    label = row['label']
    filename = os.path.join(dataset,name)
    
    try:
        print(name, label)
        sound, sr = lb.load(filename, sr=None)
        sound = lb.util.pad_center(sound, 30*sr)
        #sound_aug = additive_noise(noisetypes, noiselist, noisesnr, numnoise, audio=sound, noisecat=None)
        
        concat,concat_norm = normalization(sound,sr,scaler)
        img_train.append(concat)
        label_train.append(to_hot_one[label])

    except:
        print('None')

aHJPsm6esHeW5apkd8KAFdia9zj2.wav positive
ptuwF1pj4ibIoVuZmt4nKb5naiZ2.wav positive
EbWP0XeMpnbrTsljro0dOGKIell1_heavy.wav positive
Ued1DKV2NkUnzWjJcPMxA6Q261r1_heavy.wav positive
ZaeW7vuikQgf7984JVqXYOtUQMs2.wav positive
gDBsK8LcY0Om4DujIs4lFSsktWu1_heavy.wav positive
M9CdGPOCUFPWV7KnLgvStPgjLrF3_heavy.wav positive
GBwlY1I67YdQ47ceNFdpK87kEr33.wav positive
RmWwhKI8kjPi5qN9H740rfm0jHA2_heavy.wav positive
Fi2lLmV3LHR5brvXs0z1JSKDvnI2.wav positive
8JuoYlwoiUgaSSUUKQnB7F7fG5S2_heavy.wav positive
KFQ0ksPwkDcLUUjbVBYWGWr9ei62_heavy.wav positive
DRoZujDy6CWVFL0UefbHEgCcD2l2.wav positive
o8LJfYAChnd5Jv0WVa6Ni7jpHf92.wav positive
75OYElXmxdQTQne9breAIqnSo7Z2_heavy.wav positive
rB4ewmXSc9P5IPhLffnBasjYU1s2_heavy.wav positive
NVzhTN4CXebH2lQVFkBTyl6ue5h2_heavy.wav positive
BVcBPtxcv3cqEdIkhjM6YenlWtb2.wav positive
46bIeJk9J8XrD4ND2IPRMQzjJBt2_heavy.wav positive
HK3FfcI5SyXc7CtQMaQiMQ2ia1n2_heavy.wav positive
saaw7uHQJQesN9AmgKyXNFAaXB82.wav positive
etg5L3iyQ3fQAyyn4uZ9jYuoUf13_heavy.wav positiv

6yut3EO9tXPCCY8O0VDWA5PCWYN2_heavy.wav positive
P25gmf6a64UuD7XtIDeL9VQMzXb2_heavy.wav positive
ELTqUBJoR4MPBzMZH3yL6yLMc283_heavy.wav positive
G5kUP2QOpVSyjK6Q9rzblmTU9vW2.wav positive
UhYCvhOCn2erbzJc5faN4ek3D1I2_heavy.wav positive
dRk3UOvfOMXyuSEddsNcpIqqN8u2_heavy.wav positive
oPLTYiEHMFTuruwTXpbRPnlz2mr1_heavy.wav positive
gwg5V9vEkMYvDqW38tY5b82qYoi2.wav positive
b7mMUQm5bObj1jwGFaNyaR07pt83.wav positive
RcKXXiJlW3ZJC73zJPmEsrxSJVV2_heavy.wav positive
kJgpo7TKalNlTkZBRs2KHyPc4us1_heavy.wav positive
BG1rEPoPYKUAHCYVzPrcB3I6VUn2_heavy.wav positive
M9CdGPOCUFPWV7KnLgvStPgjLrF3.wav positive
4sjgUTKdPXcgwHbPcqEtB6WA7XB3.wav positive
hVZQcrz6cfaOMfL3jpRqJBUiumF2.wav positive
TLuM4QmyP5QZyFdkcbXWeNhWlsj1.wav positive
333NjqA1TfZJuICEdXSkPhVz0LA3_heavy.wav positive
vFBX63V7KXfdR1mQPqaL5I33Yhb2.wav positive
8Ul16g4L9nP9lcp16BKh1X6cfhb2_heavy.wav positive
gcBPC2UzklTmymsRwuBcQlCsb7k2_heavy.wav positive
MF5f99sONpcYl5fYsVzh32XwSmm2_heavy.wav positive
OSva8KJZQuZDquBrWQRFdEOtKFc2.wav positiv

v4G9xCIR9DdWAiFRSNd4XbwPUP32_heavy.wav positive
ygiPwZYsKFetOejhxWXB6rKSHAf2_heavy.wav positive
u7b9tNSshrPFYAno8H85DgImH1B3_heavy.wav positive
Ft82dZ2O4PMuq64dPIEVik9upMr2.wav positive
CRrk6OagJdebILgQWEGKDZXzi7z1.wav positive
t5wo2qvXdBehDj8JuF9pGWbb3Rt1_heavy.wav positive
xyNunpsL01N3hkRYCo7pfIkSDgf2.wav positive
VXBwsSJqsAdPR3wRmp76XFziA9f1.wav positive
bx564wHvi9d0HyJTtdi0haZuU493_heavy.wav positive
eY44bQRKksTntBarwq9fRkPkA8N2.wav positive
2tzjnTx3muV7J1n2nz8oh93WNhD3.wav positive
cRKKIIXiXQUgy3HHRhOoXl3Cfn93.wav positive
BauEFEXAglbZSSgJEYGF4FqluZG3_heavy.wav positive
ojZkyl3HYKh3UwCn9h1hOCT7Z4q2_heavy.wav positive
padXEGczMRRKmLRVc68rEKI142B3_heavy.wav positive
fWUMPPgD78VZinzLLLwqbyTysJa2_heavy.wav positive
FLNZB0Us1PgOdXLJUC4tmsvRrH43_heavy.wav positive
JUTgBmtV3PabkG7XQANi7iHlUBy2.wav positive
92TIULj0fjS6dRuCgnsgGD1SMnl2_heavy.wav positive
khBKxUEe1rQxPqpo4pGylAZjsEt1.wav positive
LasObOSE2kQYWhWkA7WimXxhwUE2.wav positive
ByJgkN2PApVkDJfUN5wp1gUuxt52.wav positive
tkM8UQsV2h

mmg71cttF0ez7wdeTZeCVFw7GV32.wav positive
YQzhlTCYjlgentYYTgVEDgNPapN2_heavy.wav positive
YA7NFQZSqwUZUJGPAh5tRe22woB2.wav positive
6T43bddKoKfG7MwnJWvrPZSsyrc2_heavy.wav positive
TaoyZAahOzRoDQRxb0DtDZh8Opa2.wav positive
y1UXzVjBZdeUxZ6fEInvyIAAS323_heavy.wav positive
8EUkoDkeGddD0Vcnwu35XuxOhDt1_heavy.wav positive
85iNcV2z6MNT3FrygbEC4fuGrIu1.wav positive
cQEUPCFkSBfwuTsWap02ban51g82_heavy.wav positive
iZa9QuNqinWMeMTdwFstfYRpw4J3_heavy.wav positive
ebtxeRmhQhQ78ZtgSjN7g637e6v1_heavy.wav positive
akdaeGnDlhgMKnIjAhpslMiMJO43_heavy.wav positive
CRrk6OagJdebILgQWEGKDZXzi7z1_heavy.wav positive
wfrSZJ0FrZMpM33MF8nQU1fiTCI2_heavy.wav positive
VXBwsSJqsAdPR3wRmp76XFziA9f1_heavy.wav positive
DT3eTAupOCRU40MK7cj8ZhzCoB53.wav positive
wRIACcPu2dS0xiHj6O7O4eJXKT53_heavy.wav positive
DRoZujDy6CWVFL0UefbHEgCcD2l2_heavy.wav positive
rGRvBhXqS0RSBDt1aVWxqfZc0i92.wav positive
A5KW9PXraNgckln1gnNaJnX6DvB2.wav positive
cf4RgKotFbYPn2HsLdxHyiChY3I2.wav positive
f9LON5JU5reyllM213wpaoJqUSu2.wav positiv

c98b76a0-089e-4552-8649-2804e2d95403.wav positive
7f8e60f4-5cbd-4ec6-bc73-e8ab2b6c3abd.wav positive
6223db52-6c9e-4604-b9f6-bfa4408a1c2d.wav positive
96678e97-6a39-4932-9c5d-13981167bbd8.wav positive
197fd3fd-2c59-41b3-836a-8c9d51b3fbb0.wav positive
fc09bf22-b83e-4f75-9db0-0d1180019ac2.wav positive
eb74838f-26b9-457e-a4f6-3fff63ebc270.wav positive
a953b5a9-6a16-4be7-88a8-1d94d5d2c686.wav positive
9cf8103f-e52d-4020-a08c-0ebd93e4b557.wav positive
76a03d41-09de-4d00-b1e6-0cedf0b3dbda.wav positive
7bff1a51-ef9b-461d-9d5f-4538d74b0fd9.wav positive
5fd93d38-c7aa-49db-bedb-f6913fcbb829.wav positive
1d6f3f96-fb43-49e6-8c38-70586a9dc52f.wav positive
099f9c59-3206-438d-8fd5-be59c5337afa.wav positive
684c1be1-e790-4f4f-a84a-ab7834310c5b.wav positive
09115490-33a9-461c-9437-d7f71be057b0.wav positive
e84f513a-6a97-4bdf-8a2e-a152ea325bc8.wav positive
3ff2bbaa-b561-46f2-b908-5f9b72758311.wav positive
6a641bd4-4cfe-4327-9413-4dd7524c9cf2.wav positive
4d230bec-1056-49ca-b274-c0825b924824.wav positive


28027420-7148-4f4f-a4d1-0884389bc6df.wav positive
d2c91dbb-f9d3-4d93-ac4d-1b0245631c82.wav positive
ffbca476-8b35-4797-bc8a-b7f0a2f24b55.wav positive
1c829cee-09e7-4c71-8148-34250eb07ecb.wav positive
9254bd84-4c4e-40cf-94d0-47a0bdf550ce.wav positive
05f2d808-7044-4c4b-8fbd-baff611383da.wav positive
c261cdbc-fb1e-4444-8b68-33c920047d8c.wav positive
92b65489-6ae0-4075-a1d6-60cc11118cf5.wav positive
edd0f863-340f-4e5b-b4ce-a0ef86db8d9a.wav positive
34c2c68f-3728-4b17-91a0-2700c1b4fac5.wav positive
7336b77f-6873-4e2c-837d-2ef925a3d33e.wav positive
27c8a255-32ce-4162-9794-b9952945800d.wav positive
f8053fc3-bc4e-403b-b9d4-14133cfe83a6.wav positive
2e4ca5b9-2e1f-4e81-b830-c75ae040dfb4.wav positive
2df9d90f-7616-482f-ae5a-e0c8d3ee4289.wav positive
13b55d58-ad6d-4cb4-933e-80622a4ff240.wav positive
ff68cc55-6fb0-4d41-831f-3642511ad303.wav positive
32ba514c-b6e5-45a5-a62d-a4555084e3ed.wav positive
4b910657-348b-4a87-bf93-257fd2081cfa.wav positive
8b8c5abd-23e2-414c-8ffc-2246bc508f19.wav positive


e7bb4543-bee0-49da-b7e6-3179d85a0b76.wav positive
f22ddcf7-6fc8-4f44-b618-65f8b5a5c74c.wav positive
e73d5a4e-b7cc-416b-8103-8aabacf3b8fe.wav positive
9031d3b2-4019-403f-9c71-f387e6b60a82.wav positive
25b750e5-8a76-4c13-9fd8-851e45d1b5ed.wav positive
9b58b701-9384-4216-ab49-c6373b6dd8b4.wav positive
6eaee903-0df8-4a22-b0c6-792d838369fa.wav positive
4fa54167-06c6-485a-a7b5-4b3696aa1393.wav positive
658daf16-7fc3-4640-911e-81dac60a2df9.wav positive
d90f21db-93fc-4e29-b023-8c2fd6a02931.wav positive
c1d86c76-5622-4dbb-9da1-65aebf6a0f0b.wav positive
32a150f7-8a0a-4c5e-b156-23552e4d58de.wav positive
21db7316-4810-4156-8892-fbd4620867d4.wav positive
0bcea474-8e87-40e5-a071-a270d58997db.wav positive
bae0c88f-852c-4f3f-8141-d037c777459e.wav positive
dc17e768-40f6-462e-99ad-80292586ac3a.wav positive
3eb99a71-eda9-4678-a328-713893887d04.wav positive
03d30b8b-03f8-4089-bda8-0e14deec7d82.wav positive
70450999-745c-43ec-b117-3eefa1b10476.wav positive
48ff5e27-832e-4450-9ea5-288a0e6967db.wav positive


141ad4b1-d9b2-407d-9e48-913915c02b74.wav positive
f18061ed-dc5a-4d84-b5d6-293b374792d0.wav positive
210860ed-f80a-4ae2-ae9c-2a67ff637388.wav positive
d57e15cf-e837-4e9e-a777-c3ee24fcb72b.wav positive
ed4ba184-bc38-4372-84c0-62485a79ec1b.wav positive
e397e500-7d0f-4a91-86f7-1111a3ddebe1.wav positive
ccdb32af-58f4-4718-9f6e-eec7e0020382.wav positive
46c6c8ef-8b3d-42a1-b4a1-e86e6aeb1664.wav positive
450dbdc0-e479-43a2-b3d2-8e51b8253649.wav positive
0eba0c0a-5609-4777-903d-b15292798250.wav positive
73b29e0f-daed-4b90-9ab8-5f33065f23a4.wav positive
ab5f091d-dfc6-4477-b584-ad2a8c1e804e.wav positive
e6dfe942-2d7a-4fb5-8273-6e1a3a82cc37.wav positive
e339c45d-998d-4f9b-a8f4-300a63985aa2.wav positive
f4e4e818-c210-4c9e-bdca-c1cf1af1e144.wav positive
a9065bbb-d564-489d-81e9-34920ba44eab.wav positive
5b5e2616-1445-4c2c-a1b3-db946cbd145a.wav positive
e5daecbd-22de-4aac-98aa-2c42b9663efe.wav positive
8e4d3ab1-5e08-424f-9146-054314535ad5.wav positive
dc994f21-ea5e-4236-bc48-2fd980feb2f8.wav positive


40de29df-7881-4cbf-9339-809f6a48ba39.wav positive
bcaf041b-32c6-4c56-b9a3-1d76d1afc99f.wav positive
8b50115d-5e51-4fa0-9521-f6ecc0b6527d.wav positive
8cd3924c-58ef-44fa-8080-4fca62aa05f4.wav positive
e3244803-32bb-4706-b14a-0bfb384b96de.wav positive
b7f28b1a-640e-4d12-9586-18b5e563ec8c.wav positive
dea52ae5-cb28-4926-a15e-9d1f0eb73115.wav positive
b138e68b-b8a7-4144-af49-57cb00f9eb6d.wav positive
a7a5c702-df32-41e5-a129-a43ec576a480.wav positive
cf38eea8-2307-4f24-88ad-1abbedb59b25.wav positive
f7a10fb3-b805-4ed0-8a93-9b0851efe78e.wav positive
2018bfea-0071-4aca-8d45-18dac4d65199.wav positive
8e40d48d-8f5d-4892-8103-451b2679283f.wav positive
054ba6b6-35c9-4b73-9016-0a132bb6d166.wav positive
46e84354-e27c-496f-ae99-fb18651aa0fc.wav positive
20602da6-a92f-40ad-81d2-1688f5f1fcde.wav positive
015ec831-e765-44ee-aefc-10dfd95dec78.wav positive
bfa6b379-56e3-4144-ac7c-17d7269e9865.wav positive
0e2f8fe3-12aa-464f-a4cc-312efa425b98.wav positive
9e7dca84-3540-447a-9446-9743d887c9aa.wav positive


train_241.wav negative
train_272.wav positive
train_060.wav positive
train_192.wav negative
train_202.wav positive
train_177.wav negative
train_093.wav negative
train_158.wav positive
train_077.wav negative
train_219.wav negative
train_285.wav positive
train_025.wav negative
train_176.wav positive
train_053.wav negative
train_240.wav positive
train_166.wav negative
train_022.wav negative
train_117.wav negative
train_110.wav negative
train_101.wav negative
train_211.wav negative
train_209.wav negative
train_028.wav negative
train_010.wav negative
train_151.wav negative
train_096.wav negative
train_258.wav negative
train_172.wav negative
train_037.wav negative
train_059.wav negative
train_107.wav positive
train_048.wav negative
train_027.wav negative
train_045.wav negative
train_086.wav positive
train_142.wav positive
train_201.wav negative
train_116.wav negative
train_054.wav positive
train_284.wav negative
train_075.wav positive
train_197.wav negative
train_255.wav positive
train_092.w

In [14]:
np.save(os.path.join('data',exp,'img_train.npy'),img_train)
np.save(os.path.join('data',exp,'label_train.npy'),label_train)

In [35]:
img_train

[tensor([[-0.1814,  0.0391,  0.0039,  ..., -0.1062,  0.0801,  0.0018],
         [-0.1772,  0.0371, -0.0026,  ..., -0.1156,  0.0793,  0.0143],
         [-0.1699,  0.0294,  0.0041,  ..., -0.1221,  0.0727,  0.0195],
         ...,
         [-0.1850,  0.0203,  0.0679,  ..., -0.0812,  0.0108, -0.1188],
         [-0.1772,  0.0226,  0.0742,  ..., -0.0797, -0.0019, -0.1349],
         [-0.1675,  0.0227,  0.0821,  ..., -0.0579, -0.0074, -0.1505]]),
 tensor([[-0.1026,  0.0739, -0.0735,  ..., -0.1584,  0.0782,  0.0073],
         [-0.0969,  0.0717, -0.0735,  ..., -0.1919,  0.0752, -0.0114],
         [-0.0983,  0.0697, -0.0725,  ..., -0.1935,  0.0713, -0.0173],
         ...,
         [-0.1185,  0.0562,  0.0175,  ..., -0.1587,  0.0852,  0.0024],
         [-0.1136,  0.0576,  0.0116,  ..., -0.1602,  0.0855,  0.0144],
         [-0.0734,  0.0656,  0.0496,  ..., -0.1312,  0.0156, -0.1062]]),
 tensor([[-0.1506,  0.0479, -0.0899,  ..., -0.0425,  0.1215,  0.0660],
         [-0.1500,  0.0492, -0.0887,  ..., -0

In [15]:
img_train = np.load(os.path.join('data',exp,'img_train.npy'))
label_train = np.load(os.path.join('data',exp,'label_train.npy'))

In [16]:
label_train.shape

(1762,)

## Extracting Devel Data Feature

In [17]:
df_devel = pd.read_csv('csv_files/experiment_data/devel.csv')
df_devel.head()

,filename,label
0,ir3jqGSKT2ManVTAAzj6Ew7orBD2_heavy.wav,positive
1,ZzKpUtFGzsVEK7E1wsVGRftKdx93.wav,positive
2,TkOj2GRygbe2J9HxfMWk02A4e7v1.wav,positive
3,ottC8aetxhRQfKD00RCJc5LWS622_heavy.wav,positive
4,IFMid2LmeCTSQAFaL8wTSUg20No2.wav,positive


In [ ]:
img_devel = []
label_devel = []

to_hot_one = {'positive':0, 'negative':1}

for index,row in df_devel.iterrows():
    name = row['filename']
    label = row['label']
    filename = os.path.join(dataset,name)
    
    try:
        print(name, label)
        sound, sr = lb.load(filename, sr=None)
        sound = lb.util.pad_center(sound, 30*sr)
        #sound_aug = additive_noise(noisetypes, noiselist, noisesnr, numnoise, audio=sound, noisecat=None)
        
        concat,concat_norm = normalization(sound,sr,scaler)
        img_devel.append(concat)
        label_devel.append(to_hot_one[label])
        
    except:
        print('None')

ir3jqGSKT2ManVTAAzj6Ew7orBD2_heavy.wav positive
ZzKpUtFGzsVEK7E1wsVGRftKdx93.wav positive
TkOj2GRygbe2J9HxfMWk02A4e7v1.wav positive
ottC8aetxhRQfKD00RCJc5LWS622_heavy.wav positive
IFMid2LmeCTSQAFaL8wTSUg20No2.wav positive
xYCukI9rgMeslwjIYWheFtyZ6zk1_heavy.wav positive
Iz9X0QRimVT2R7AGhD6jZM0ryPp1.wav positive
CGXh77ubexNuv0UUmOHVCnAgurY2.wav positive
BDygu23lyUbBq2NlqfzRxGoMm9B2_heavy.wav positive
cd57GKgW74bBSqmusnCkc8bXujs2.wav positive
2lsppspH44WH3Scc1ojisHaXJzp1_heavy.wav positive
Hge7rEPBdLcy4ckCtvUFfl4p0Ne2_heavy.wav positive
EUjlCZyR0xMbOH31Je8knI3UdKO2.wav positive
LF7wSEDpWafkz5bwtxnFuho9iWz1.wav positive
y0TkG5sfyhadW3VSct50RTl3HN02.wav positive
t5wo2qvXdBehDj8JuF9pGWbb3Rt1.wav positive
rB4ewmXSc9P5IPhLffnBasjYU1s2.wav positive


In [ ]:
np.save(os.path.join('data',exp,'img_devel.npy'),img_devel)
np.save(os.path.join('data',exp,'label_devel.npy'),label_devel)

In [ ]:
img_devel = np.load(os.path.join('data',exp,'img_devel.npy'))
label_devel = np.load(os.path.join('data',exp,'label_devel.npy'))

## Extracting Test Data Feature 

In [ ]:
df_test = pd.read_csv('csv_files/experiment_data/test.csv')
df_test.head()

In [ ]:
img_test = []
label_test = []

to_hot_one = {'positive':0, 'negative':1}

for index,row in df_test.iterrows():
    name = row['filename']
    label = row['label']
    filename = os.path.join(dataset,name)
    
    try:
        print(name, label)
        sound, sr = lb.load(filename, sr=None)
        sound = lb.util.pad_center(sound, 30*sr)
        
        concat,concat_norm = normalization(sound,sr,scaler)
        img_test.append(concat)
        label_test.append(to_hot_one[label])
    except:
        print('None')

In [ ]:
np.save(os.path.join('data',exp,'img_test.npy'),img_test)
np.save(os.path.join('data',exp,'label_test.npy'),label_test)

In [ ]:
img_test = np.load(os.path.join('data',exp,'img_test.npy'))
label_test = np.load(os.path.join('data',exp,'label_test.npy'))

## Load Data

In [ ]:
img_train = np.load(os.path.join('data',exp,'img_train.npy'))
label_train = np.load(os.path.join('data',exp,'label_train.npy'))

img_devel = np.load(os.path.join('data',exp,'img_devel.npy'))
label_devel = np.load(os.path.join('data',exp,'label_devel.npy'))

img_test = np.load(os.path.join('data',exp,'img_test.npy'))
label_test = np.load(os.path.join('data',exp,'label_test.npy'))

In [ ]:
img_train = np.array(img_train)
label_train = np.array(label_train)

img_devel = np.array(img_devel)
label_devel = np.array(label_devel)

img_test = np.array(img_test)
label_test = np.array(label_test)

In [ ]:
print(img_train.shape)
print(img_devel.shape)
print(img_test.shape)

In [ ]:
if os.path.exists(os.path.join('data',exp,'array'))==False:
    os.mkdir(os.path.join('data',exp,'array'))

## Save data array

In [ ]:
np.save(os.path.join('data',exp,'array','img_train.npy'),img_train)
np.save(os.path.join('data',exp,'array','label_train.npy'),label_train)

np.save(os.path.join('data',exp,'array','img_devel.npy'),img_devel)
np.save(os.path.join('data',exp,'array','label_devel.npy'),label_devel)

np.save(os.path.join('data',exp,'array','img_test.npy'),img_test)
np.save(os.path.join('data',exp,'array','label_test.npy'),label_test)

In [ ]:
img_train = np.load(os.path.join('data',exp,'array','img_train.npy'))
label_train = np.load(os.path.join('data',exp,'array','label_train.npy'))

img_val = np.load(os.path.join('data',exp,'array','img_devel.npy'))
label_val = np.load(os.path.join('data',exp,'array','label_devel.npy'))

img_test = np.load(os.path.join('data',exp,'array','img_test.npy'))
label_test = np.load(os.path.join('data',exp,'array','label_test.npy'))

## Augmented using SMOTE

In [ ]:
oversample = SMOTE()
img_train_smote, label_train_smote = oversample.fit_resample(np.reshape(img_train,[img_train.shape[0],img_train.shape[1]]), label_train)

In [ ]:
oversample = SMOTE()
img_val_smote, label_val_smote = oversample.fit_resample(np.reshape(img_val,[img_val.shape[0],img_val.shape[1]]), label_val)

In [ ]:
np.save(os.path.join('data',exp,'array','img_train_smote.npy'),img_train_smote)
np.save(os.path.join('data',exp,'array','label_train_smote.npy'),label_train_smote)

np.save(os.path.join('data',exp,'array','img_devel_smote.npy'),img_val_smote)
np.save(os.path.join('data',exp,'array','label_devel_smote.npy'),label_val_smote)